In [1]:
from tkinter import *
import random
import json
import pickle
import numpy as np
import tensorflow as tf
import nltk
from nltk.stem import WordNetLemmatizer
from keras.models import load_model

BG_GRAY = "#ABB2B9"
BG_COLOR = "#17202A"
TEXT_COLOR = "#EAECEE"

FONT = "Helvetica 14"
FONT_BOLD = "Helvetica 13 bold"

# Constants and setup for the chatbot
GIBBERISH_THRESHOLD = 0.1
FALLBACK_THRESHOLD = 0.2

bot_name = "JYPD Bot"

# Initialize the WordNetLemmatizer for lemmatization
lemmatizer = WordNetLemmatizer()

# Load the initial intents data from a JSON file
intents = json.load(open('michade_farms.json'))

# Load the previously pickled words and classes lists
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

# Load the trained model
try:
    model = load_model("my_model.keras")
except Exception as e:
    print(f"Error loading the model: {e}")
    # Handle the error or exit the program

class ChatContext:
    def __init__(self):
        self.current_intent = None
        self.user_input_history = []
        self.last_response = None
        self.user_name = None
        self.user_location = None
        self.user_preferences = {}

# Function to handle gibberish responses when the chatbot doesn't understand the user
def handle_gibberish_response():
    return "Mich Bot: I'm sorry, but I couldn't understand your input. Please try again."

# Function to handle fallback responses when the chatbot doesn't understand the user
def handle_fallback_response(context):
    response = "Mich Bot: Sorry, I don't understand. Would you like to talk to the support team?"
    user_response = input(f"{context.user_name}: ").lower()
    if user_response in ["yes", "yeah", "ok", "sure"]:
        # Provide contact information for the support team
        support_email = "info@michadefarms.com.ng"
        support_phone = "+2348037149761"
        response = (f"Mich Bot: Sure! You can reach our support team via email at {support_email} "
                    f"or by phone at {support_phone}.")
    elif user_response in ["no", "nope"]:
        response = "Mich Bot: No problem. Would you like to ask me any other questions?"
    else:
        response = "Mich Bot: I'm sorry, please respond with 'yes' or 'no'."
    
    # Update context with the fact that the chatbot didn't understand the user's input
    context.last_response = "I don't understand. Please try again."
    return response

# Function to clean up a sentence by tokenizing and lemmatizing the words
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

# Function to convert a sentence into a bag of words representation
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

# Function to predict the class (intent) of a given sentence using the trained model
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

# Function to get a random response for a given intent from the intents data
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result


class ChatApplication:
    def __init__(self):
        self.window = Tk()
        self._setup_main_window()
        self.context = ChatContext()  # Create an instance of ChatContext
        self.last_bot_response = None  # Add a variable to store the last bot response

    def run(self):
        self.window.mainloop()

    def _setup_main_window(self):
        self.window.title("JPYD ChatBot")
        self.window.resizable(width=False, height=False)
        self.window.configure(width=1200, height=700, bg=BG_COLOR)

        # head label
        head_label = Label(self.window, bg=BG_COLOR, fg=TEXT_COLOR,
                           text="Your agriculture friend", font=FONT_BOLD, pady=10)
        head_label.place(relwidth=1)

        # tiny divider
        line = Label(self.window, width=450, bg=BG_GRAY)
        line.place(relwidth=1, rely=0.07, relheight=0.012)

        # text widget
        self.text_widget = Text(self.window, width=20, height=2, bg=BG_COLOR, fg=TEXT_COLOR,
                                font=FONT, padx=5, pady=5)
        self.text_widget.place(relheight=0.745, relwidth=1, rely=0.08)
        self.text_widget.configure(cursor="arrow", state=DISABLED)

        # scroll bar
        scrollbar = Scrollbar(self.text_widget)
        scrollbar.place(relheight=1, relx=0.974)
        scrollbar.configure(command=self.text_widget.yview)

        # bottom label
        bottom_label = Label(self.window, bg=BG_GRAY, height=80)
        bottom_label.place(relwidth=1, rely=0.825)

        # message entry box
        self.msg_entry = Entry(bottom_label, bg="#2C3E50", fg=TEXT_COLOR, font=FONT)
        self.msg_entry.place(relwidth=0.74, relheight=0.06, rely=0.008, relx=0.011)
        self.msg_entry.focus()
        self.msg_entry.bind("<Return>", self._on_enter_pressed)

        # send button
        send_button = Button(bottom_label, text="Send", font=FONT_BOLD, width=20, bg=BG_GRAY,
                             command=lambda: self._on_enter_pressed(None))
        send_button.place(relx=0.77, rely=0.008, relheight=0.06, relwidth=0.22)


        # send button
        send_button = Button(bottom_label, text="Send", font=FONT_BOLD, width=20, bg=BG_GRAY,
                             command=lambda: self._on_enter_pressed(None))
        send_button.place(relx=0.77, rely=0.008, relheight=0.06, relwidth=0.22)

    def _on_enter_pressed(self, event):
        msg = self.msg_entry.get()
        self._insert_message(msg, "You")
        self.handle_user_input(msg.lower())  # Call the chatbot handling method

    def handle_user_input(self, user_input):
        # Adapted handling method for the chatbot
        if user_input.lower() in ["quit", "exit"]:
            response = "Mich Bot: Goodbye! Chatbot is now exiting."
            self.text_widget.configure(state=NORMAL)
            self.text_widget.insert(END, response + "\n\n")
            self.text_widget.configure(state=DISABLED)
            self.msg_entry.config(state=DISABLED)
            return

        # Predict the intent and get the response
        ints = predict_class(user_input)

        # Identify gibberish responses
        if not ints or float(ints[0]['probability']) < GIBBERISH_THRESHOLD:
            response = handle_gibberish_response()
        else:
            # Identify fallback responses
            if float(ints[0]['probability']) < FALLBACK_THRESHOLD:
                response = handle_fallback_response(self.context)
            else:
                res = get_response(ints, intents)  # Provide the 'intents' variable
                response = f"{bot_name}: {res}\n\n"
                self.last_bot_response = response  # Store the bot response

        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, response)
        self.text_widget.configure(state=DISABLED)
        self.text_widget.see(END)
        self.msg_entry.delete(0, END)

    def _insert_message(self, msg, sender):
        if not msg:
            return

        self.msg_entry.delete(0, END)
        msg1 = f"{sender}: {msg}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg1)
        self.text_widget.configure(state=DISABLED)

        # Insert the last bot response without making another call to get_response
        if self.last_bot_response:
            self.text_widget.insert(END, self.last_bot_response)
            self.last_bot_response = None  # Reset the variable

        self.text_widget.see(END)

if __name__ == "__main__":
    app = ChatApplication()
    app.run()




1/1 [==============================] - 0s 25ms/step
